In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import scipy.io as sio

In [2]:


# === Parameters ===
image_size = (48, 48)  # Resize all images to this
dataset_path = 'Compound_Emotion'  # <- Change this

# === Step 1: Load images and labels ===
X = []
y = []
label_names = sorted(os.listdir(dataset_path))
label_to_int = {label: idx for idx, label in enumerate(label_names)}

for label in label_names:
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue
    for fname in os.listdir(folder):
        try:
            fpath = os.path.join(folder, fname)
            img = Image.open(fpath).convert('L')  # Grayscale
            img = img.resize(image_size)
            img_np = np.array(img, dtype=np.float32) / 255.0  # Normalize
            X.append(img_np)
            y.append(label_to_int[label])
        except Exception as e:
            print(f"Error reading {fpath}: {e}")

X = np.stack(X)  # (n_samples, 48, 48)
X = X[:, :, :, np.newaxis]  # (n_samples, 48, 48, 1)
y = np.array(y, dtype=np.int32)
    
# === Step 2: Split into Train/Val/Test ===
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, stratify=y_temp, random_state=42)
# This results in 80% train, 10% val, 10% test

# === Step 3: Save to .mat ===
sio.savemat('compound_emotion_cleaned.mat', {
    'X_train': X_train,
    'y_train': y_train,
    'X_val': X_val,
    'y_val': y_val,
    'X_test': X_test,
    'y_test': y_test,
    'label_names': np.array(label_names)
})

print("Saved as 'compound_emotion_cleaned.mat'")


Saved as 'compound_emotion_cleaned.mat'
